![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train Legal Assertion

In [0]:
from johnsnowlabs import * 

# Data Prep

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Legal/data/assertion_fin.csv
dbutils.fs.cp("file:/databricks/driver/assertion_fin.csv", "dbfs:/Finance") 

Out[2]: True

In [0]:
import pandas as pd

training_df = pd.read_csv('/dbfs/Finance/assertion_fin.csv')

In [0]:
training_data = spark.createDataFrame(training_df)
training_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
task_id| sentence|tkn_start|tkn_end| chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
 1|The Swedish East ...| 1| 4|Swedish East Indi...| ORG| PAST|
 1|The Swedish East ...| 6| 8|Svenska Ostindisk...| ALIAS| PAST|
 1|The Swedish East ...| 10| 10| SOIC| ALIAS| PAST|
 1|The Swedish East ...| 14| 14| Gothenburg| LOC| PAST|
 1|The Swedish East ...| 15| 15| Sweden| LOC| PAST|
 1|The Swedish East ...| 17| 17| 1731| DATE| PAST|
 1|The Swedish East ...| 25| 25| China| LOC| PAST|
 1|The Swedish East ...| 28| 29| Far East| LOC| PAST|
 1|The venture was i...| 9| 12|Dutch East India ...| ORG| PAST|
 1|The venture was i...| 15| 18|British East Indi...| ORG| PAST|
 1|This made Gothenb...| 2| 2| Gothenburg| LOC| PAST|
 1|Trade with China ...| 2| 2| China| LOC| PAST|
 1|Trade with China ...| 11| 11| Sweden| LOC| PAST|
 1|The Chinese cultu...| 34| 34| Sweden| LOC| PAST|
 1|The company folde...| 4| 4| 1813| DATE| PAST|
 1|nevertheless, it ...| 11| 11| Gothenburg| LOC| PAST|
 1|Background Sweden...| 16| 17| East India| LOC| PAST|
 1|The royal privile...| 5| 8|Swedish East Indi...| ORG| PAST|
 1|The royal privile...| 9| 9| SOIC| ALIAS| PAST|
 1|The royal privile...| 27| 28| East India| LOC| PAST|
+-------+--------------------+---------+-------+--------------------+------+---------------+
only showing top 20 rows

In [0]:
training_data.printSchema()

root
-- task_id: long (nullable = true)
-- sentence: string (nullable = true)
-- tkn_start: long (nullable = true)
-- tkn_end: long (nullable = true)
-- chunk: string (nullable = true)
-- entity: string (nullable = true)
-- assertion_label: string (nullable = true)

In [0]:
%time training_data.count()

CPU times: user 442 ms, sys: 177 ms, total: 619 ms
Wall time: 2min 14s
Out[6]: 8050

In [0]:
(train_data, test_data) = training_data.randomSplit([0.9, 0.1], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 8050
Test Dataset Count: 831

In [0]:
train_data.show()

+-------+--------------------+---------+-------+--------------------+------+---------------+
task_id| sentence|tkn_start|tkn_end| chunk|entity|assertion_label|
+-------+--------------------+---------+-------+--------------------+------+---------------+
 1|2.5 tonnes) and t...| 11| 11| Gothenburg| LOC| PAST|
 1|2.5 tonnes) and t...| 34| 34| Sweden| LOC| PAST|
 1|= Early attempts ...| 9| 11| Swedish East India| ORG| PAST|
 1|= Early attempts ...| 20| 21| Willem Usselincx| PER| PAST|
 1|= Sweden after th...| 1| 1| Sweden| LOC| PAST|
 1|= The Royal chart...| 9| 12|Henrik König & Co...| ORG| PAST|
 1|= The Royal chart...| 39| 42| Cape of Good Hope| LOC| PAST|
 1|= The Royal chart...| 46| 46| Japan| LOC| PAST|
 1|= The Royal chart...| 70| 70| Gothenburg| LOC| PAST|
 1|= The Royal chart...| 79| 79| Gothenburg| LOC| PAST|
 1|= The first octro...| 8| 8| directors| ROLE| PAST|
 1|= The first octro...| 11| 11| SOIC| ORG| PAST|
 1|= The first octro...| 13| 14| Henrik König| PER| PAST|
 1|= The first octro...| 18| 19| Frans Bedoire| PER| PAST|
 1|= The first octro...| 23| 23| Stockholm| LOC| PAST|
 1|= The second octr...| 10| 10| 1746| DATE| PAST|
 1|= The second octr...| 11| 11| 1766| DATE| PAST|
 1|= The second octr...| 13| 13| 1786| DATE| PAST|
 1|= The second octr...| 38| 38| Canton| LOC| PAST|
 1|A couple of weeks...| 16| 19|Anders Plomgren &...| ORG| PAST|
+-------+--------------------+---------+-------+--------------------+------+---------------+
only showing top 20 rows

# Using RoBerta Embeddings

In [0]:
roberta_embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
document = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")

chunk = nlp.Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("doc_chunk")\
    .setChunkCol("chunk")\
    .setStartCol("tkn_start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(False)

token = nlp.Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')


We save the test data in parquet format to use in `AssertionDLApproach()`.

In [0]:
assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    roberta_embeddings])

assertion_test_data = assertion_pipeline.fit(test_data).transform(test_data)

In [0]:
assertion_test_data.columns

Out[12]: ['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

In [0]:
assertion_test_data.write.mode('overwrite').parquet('/dbfs/test_data.parquet')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3850215879366020> in <module> 
 ----> 1 assertion_test_data . write . mode ( 'overwrite' ) . parquet ( '/dbfs/test_data.parquet' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 1275 self . partitionBy ( partitionBy ) 
 1276 self . _set_opts ( compression = compression ) 
 -> 1277 self . _jwrite . parquet ( path ) 
 1278 
 1279 def text ( self , path , compression = None , lineSep = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1291.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:307)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:194)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:121)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:119)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:144)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:213)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:257)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:209)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:167)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:166)
	at org.apache.spark.sql.DataFrameWriter.$anonfun$runCommand$1(DataFrameWriter.scala:1080)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:156)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:299)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:130)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:103)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:249)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:1080)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:469)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:439)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:304)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:965)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.

In [0]:
assertion_train_data = assertion_pipeline.fit(training_data).transform(training_data)
assertion_train_data.write.mode('overwrite').parquet('/dbfs/train_data.parquet')

In [0]:
assertion_train_data.columns

Out[16]: ['task_id',
 'sentence',
 'tkn_start',
 'tkn_end',
 'chunk',
 'entity',
 'assertion_label',
 'document',
 'doc_chunk',
 'token',
 'embeddings']

## Graph setup

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline. 

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [0]:
graph_folder= "/dbfs/tf_graphs"

In [0]:
assertion_graph_builder =  legal.TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("assertion_label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(1200)\
    .setHiddenUnitsNumber(25)

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [0]:
scope_window = [50, 50]

assertionStatus = legal.AssertionDLApproach()\
    .setLabelCol("assertion_label")\
    .setInputCols("document", "doc_chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setEpochs(2)\
    .setStartCol("tkn_start")\
    .setEndCol("tkn_end")\
    .setMaxSentLen(1200)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('dbfs:/assertion/training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="dbfs:/test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)
    #.setValidationSplit(0.2)\    
    #.setDropout(0.1)\    

In [0]:
legal_assertion_pipeline = nlp.Pipeline(
    stages = [
    #document,
    #chunk,
    #token,
    #embeddings,
    assertion_graph_builder,
    assertionStatus])

In [0]:
training_data.printSchema()

root
-- task_id: long (nullable = true)
-- sentence: string (nullable = true)
-- tkn_start: long (nullable = true)
-- tkn_end: long (nullable = true)
-- chunk: string (nullable = true)
-- entity: string (nullable = true)
-- assertion_label: string (nullable = true)

In [0]:
assertion_train_data = spark.read.parquet('/dbfs/train_data.parquet')

In [0]:
%%time
assertion_model = legal_assertion_pipeline.fit(assertion_train_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: /dbfs/tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 4, 'feat_size': 768, 'max_seq_len': 1200, 'n_hidden': 25}
Device mapping: no known devices.
Device mapping: no known devices.
assertion_dl graph exported to /dbfs/tf_graphs/assertion_graph.pb
CPU times: user 10.6 s, sys: 2.27 s, total: 12.9 s
Wall time: 26min 15s

Checking the results saved in the log file

In [0]:
import os

log_files = os.listdir("/dbfs/assertion/training_logs/")
log_files

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-3850215879366034> in <module> 
 1 import os
 2 
 ----> 3 log_files = os . listdir ( "/dbfs/assertion/training_logs/" ) 
 4 log_files

 FileNotFoundError : [Errno 2] No such file or directory: '/dbfs/assertion/training_logs/'

In [0]:
with open("/dbfs/assertion/training_logs/"+log_files[0]) as log_file:
    print(log_file.read())

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3850215879366035> in <module> 
 ----> 1 with open ( "/dbfs/assertion/training_logs/" + log_files [ 0 ] ) as log_file : 
 2 print ( log_file . read ( ) ) 

 NameError : name 'log_files' is not defined

In [0]:
assertion_test_data = spark.read.parquet('/dbfs/test_data.parquet')

In [0]:
preds = assertion_model.transform(assertion_test_data).select('assertion_label','assertion.result')

preds.show()

+---------------+---------+
assertion_label| result|
+---------------+---------+
 PAST| [PAST]|
 PAST| [PAST]|
 PAST|[PRESENT]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PRESENT|[PRESENT]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
 PAST| [PAST]|
+---------------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()

In [0]:
preds_df["result"] = preds_df["result"].apply(lambda x: x[0] if len(x) else pd.NA)
preds_df.dropna(inplace=True)

preds_df

Out[29]:

,assertion_label,result
0,PAST,PAST
1,PAST,PAST
2,PAST,PRESENT
3,PAST,PAST
4,PAST,PAST
...,...,...
792,POSSIBLE,POSSIBLE
793,POSSIBLE,POSSIBLE
794,POSSIBLE,POSSIBLE
795,POSSIBLE,POSSIBLE


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['assertion_label'], preds_df['result']))

precision recall f1-score support

 FUTURE 0.97 0.95 0.96 123
 PAST 0.92 0.97 0.94 278
 POSSIBLE 0.98 0.97 0.97 177
 PRESENT 0.95 0.89 0.92 217

 accuracy 0.95 795
 macro avg 0.95 0.95 0.95 795
weighted avg 0.95 0.95 0.95 795

### Saving the trained model

In [0]:
assertion_model.stages

Out[31]: [TFGraphBuilderModel_d54cd897c743, FINANCE-ASSERTION_DL_9a79432c6100]

In [0]:
# Save a Spark NLP model
assertion_model.stages[-1].write().overwrite().save('/dbfs/Assertion')